<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/09_2_langgraph_Self_RAG.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Upstage RAG API (experimental)



## Environment

In [1]:
!pip install -qU langchain-upstage langchain 
!pip install -qU python-dotenv

### Environment variables

Set up environment variables 
* UPSTAGE_API_KEY


In [2]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [3]:
from openai import OpenAI

model_name = "test_model"

client = OpenAI(
    base_url="https://experimental-api.x.upstage.ai/rag-api/",
    api_key=os.environ["UPSTAGE_API_KEY"]
)

# Prompt user to decide whether to build/rebuild the index
rebuild_index = input("Do you want to build/rebuild the index? (y/n): ").strip().lower()

if rebuild_index == 'y':
    print("Building/Rebuilding the index...")
    # Upload all files in the pdfs folder
    for file in os.listdir("pdfs"):
        file_path = os.path.join("pdfs", file)
        with open(file_path, "rb") as f:
            file_response = client.files.create(
                file=f,
                purpose="assistants",
                extra_body={"model_name": model_name}
            )
        pprint(file_response)
    print("Index build/rebuild completed.")
else:
    print("Index build/rebuild skipped by the user.")

Index build/rebuild skipped by the user.


In [4]:
from pprint import pprint
from langchain_upstage import ChatUpstage as Chat
import time
for i in range(3):
    try:
        chat = Chat(model=model_name, base_url="https://experimental-api.x.upstage.ai/rag-api/")
        response = chat.invoke("what is DUS? Why is it important?")
        pprint(response.content)
        break
    except Exception as e:
        if "503" in str(e):
            print(f"503 Service Unavailable error occurred {e}. Retry {i+1}...")
            time.sleep(3)
        else:
            raise e
        
    if i == 2:
        raise Exception("Failed to get response from RAG API")

503 Service Unavailable error occurred Error code: 503 - {'id': '_', 'object': 'chat.completion', 'created': 1727378057.0, 'model': 'test_model', 'choices': [{'message': {'role': 'assistant', 'content': 'File indexing is in progress. Please try again later.'}, 'index': 0, 'finish_reason': None}]}. Retry 1...
503 Service Unavailable error occurred Error code: 503 - {'id': '_', 'object': 'chat.completion', 'created': 1727378067.0, 'model': 'test_model', 'choices': [{'message': {'role': 'assistant', 'content': 'File indexing is in progress. Please try again later.'}, 'index': 0, 'finish_reason': None}]}. Retry 2...
503 Service Unavailable error occurred Error code: 503 - {'id': '_', 'object': 'chat.completion', 'created': 1727378076.0, 'model': 'test_model', 'choices': [{'message': {'role': 'assistant', 'content': 'File indexing is in progress. Please try again later.'}, 'index': 0, 'finish_reason': None}]}. Retry 3...


Exception: Failed to get response from RAG API